In [39]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl
from bs4 import BeautifulSoup
import mechanicalsoup
import os
import yaml
import re
import cssutils
from jsmin import jsmin
from datetime import datetime

In [40]:
# https://bitbucket.org/cthedot/cssutils/issues/60/using-units-of-rem-produces-an-invalid

from cssutils import profile
profile._MACROS['length'] = r'0|{num}(em|ex|px|in|cm|mm|pt|pc|q|ch|rem|vw|vh|vmin|vmax)'
profile._MACROS['positivelength'] = r'0|{positivenum}(em|ex|px|in|cm|mm|pt|pc|q|ch|rem|vw|vh|vmin|vmax)'
profile._MACROS['angle'] = r'0|{num}(deg|grad|rad|turn)'
profile._resetProperties()

In [41]:
class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1)


In [42]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

team = config['team']
html = config['html']
css  = config['css']
js   = config['js']


In [43]:
def URLreplace(relative, extension):
    if re.match('http', relative):
        return relative
    else:
        extension = extension.upper()
        if extension == 'CSS':
            type = 'css'
        elif extension == 'JS':
            type = 'javascript'
        absolute = os.path.splitext(relative)[0] + extension
        absolute = 'https://2020.igem.org/Template:' + team + '/' + absolute + \
                        '?action=raw' + '&' + 'ctype=text/' + type
        return absolute
    

In [102]:
def uploadURL(relative, extension):
    if re.match('http', relative):
        return relative
    else:
        extension = extension.lower()
        if extension == 'html':
            relative = os.path.dirname(relative)
            if relative != '':
                relative = '/' + relative
            return 'https://2020.igem.org/wiki/index.php?title=Team:' + team + relative + '&action=edit'
        elif extension == 'css' or extension == 'js':
            extension = extension.upper()
            final = os.path.splitext(relative)[0] + extension
            return 'https://2020.igem.org/wiki/index.php?title=Template:' + team + '/' + final + '&action=edit'

In [52]:
username = 'ballaneypranav'
password = 'xWTs5wUKxb2Njx'

In [53]:
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://igem.org/Login2")
browser.select_form('form[method="post"]')
browser["username"] = username
browser["password"] = password
response = browser.submit_selected()

<input name="return_to" type="hidden" value=""/>
<input name="username" size="20" type="text" value=""/>
<input name="password" size="20" type="password" value=""/>
<input class="submit" name="Login" type="submit" value="Login"/>
<input class="link_text" name="mail_password" type="submit" value="Email me a new password"/>


In [54]:
print(response.text)

<div style='width:500px;margin:0px auto;padding:20px;background-color:white;'>You have successfully logged into the iGEM web sites.</div>


In [94]:
def iGEMupload(filename, url):
    with open(filename, 'r') as file:
        contents = file.read()
    print('Uploading ' + filename + ' to ' + url)
    browser.open(url)
    browser.select_form('form')
    browser['wpTextbox1'] = contents
    browser['wpSummary'] = 'Uploaded using an API at ' + str(datetime.now())
    browser.submit_selected()

In [93]:
# process HTML files
for filename in html:
    outfile  = 'igem/' + filename
    infile   = 'dist/' + filename
    with open(infile, 'r') as file:
        source = file.read()
        soup = BeautifulSoup(source, 'html.parser')

        css_tags = soup.findAll('link')
        for css_tag in css_tags:
            css_tag['href'] = URLreplace(css_tag['href'], 'css')
            print(css_tag['href'])

        js_tags = soup.findAll('script')
        for js_tag in js_tags:
            js_tag['src'] = URLreplace(js_tag['src'], 'js')
            print(js_tag['src'])

    with open(outfile, 'w') as f_output:
        f_output.write(str(soup))
    
    url = uploadURL(filename, 'html')
    iGEMupload(outfile, url)

https://2020.igem.org/Template:BITSPilani-Goa_India/css/indexCSS?action=raw&ctype=text/css
https://2020.igem.org/Template:BITSPilani-Goa_India/js/indexJS?action=raw&ctype=text/javascript


In [103]:
if not os.path.exists('igem/css'):
    os.makedirs('igem/css')
cssutils.ser.prefs.useMinified()
for filename in css:
    filename = 'css/' + filename
    outfile = 'igem/' + filename[:-4] + 'CSS'
    infile = 'dist/' + filename
    sheet = cssutils.parseFile(infile)
    with open(outfile, 'wb') as f:
        f.write(sheet.cssText)

    url = uploadURL(filename, 'css')
    iGEMupload(outfile, url)

Uploading igem/css/indexCSS to https://2020.igem.org/wiki/index.php?title=Template:BITSPilani-Goa_India/css/indexCSS&action=edit


In [111]:
if not os.path.exists('igem/js'):
    os.makedirs('igem/js')
for filename in js:
    filename = 'js/' + filename
    outfile = 'igem/' + filename[:-3] + 'JS'
    infile = 'dist/' + filename
    with open(infile, 'r') as f:
        script = f.read()
    
    script = jsmin(script)
    
    with open(outfile, 'w') as f:
        f.write(script)

    url = uploadURL(filename, 'js')
    iGEMupload(outfile, url)

Uploading igem/js/indexJS to https://2020.igem.org/wiki/index.php?title=Template:BITSPilani-Goa_India/js/indexJS&action=edit
